In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
pip install DeepMIMOv3 umap-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
#LWM을 하기위한 라이브러리 가져오기
import DeepMIMOv3
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

plt . rcParams [ 'figure.figsize' ]  =  [ 12 ,  8 ]  # 기본 플롯 크기 설정

In [5]:
## Load and print the default parameters
# bandwith: 0.05GHz(50MHz 대역폭 사용)

parameters = DeepMIMOv3.default_params()
pprint(parameters)

{'OFDM': {'RX_filter': 0,
          'bandwidth': 0.05,
          'selected_subcarriers': array([0]),
          'subcarriers': 512},
 'OFDM_channels': 1,
 'active_BS': array([1]),
 'bs_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([8, 4]),
                'spacing': 0.5},
 'dataset_folder': './Raytracing_scenarios',
 'dynamic_scenario_scenes': array([1]),
 'enable_BS2BS': 1,
 'enable_doppler': 0,
 'enable_dual_polar': 0,
 'num_paths': 5,
 'scenario': 'O1_60',
 'ue_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([4, 2]),
                'spacing': 0.5},
 'user_rows': array([1]),
 'user_subsampling': 1}


In [6]:
## Change parameters for the setup
# Scenario O1_60 extracted at the dataset_folder
#LWM 동적 시나리오 불러오기
#자신의 LWM 파일 위치 경로 작성
# parameters['dataset_folder'] = r'/content/drive/MyDrive/Colab Notebooks/LWM'
parameters['dataset_folder'] = r'C:\Users\dlghd\졸업프로젝트\LWM'

# scnario = 02_dyn_3p5 <- 다운받은 파일(동적시나리오)
parameters['scenario'] = 'O2_dyn_3p5'
parameters['dynamic_scenario_scenes'] = np.arange(10) #scene 0~9

# 각 사용자-기지국 채널에 대해 최대 10개 멀티패스 경로 사용
parameters['num_paths'] = 10

# User rows 1-100
parameters['user_rows'] = np.arange(100)

# Activate only the first basestation
parameters['active_BS'] = np.array([1])

parameters['activate_OFDM'] = 1

parameters['OFDM']['bandwidth'] = 0.05 # 50 MHz
parameters['OFDM']['subcarriers'] = 512 # OFDM with 512 subcarriers
parameters['OFDM']['selected_subcarriers'] = np.arange(0, 64, 1)
#parameters['OFDM']['subcarriers_limit'] = 64 # Keep only first 64 subcarriers

parameters['ue_antenna']['shape'] = np.array([1, 1]) # Single antenna
parameters['bs_antenna']['shape'] = np.array([1, 32]) # ULA of 32 elements
#parameters['bs_antenna']['rotation'] = np.array([0, 30, 90]) # ULA of 32 elements
#parameters['ue_antenna']['rotation'] = np.array([[0, 30], [30, 60], [60, 90]]) # ULA of 32 elements
#parameters['ue_antenna']['radiation_pattern'] = 'isotropic'
#parameters['bs_antenna']['radiation_pattern'] = 'halfwave-dipole'

In [7]:
print(parameters)

{'dataset_folder': 'C:\\Users\\dlghd\\졸업프로젝트\\LWM', 'scenario': 'O2_dyn_3p5', 'dynamic_scenario_scenes': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), 'num_paths': 10, 'active_BS': array([1]), 'user_rows': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), 'user_subsampling': 1, 'bs_antenna': {'shape': array([ 1, 32]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'ue_antenna': {'shape': array([1, 1]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'enable_doppler': 0, 'enable_dual_polar'

In [8]:
## Generate and inspect the dataset
import time
start = time.time()
dataset = DeepMIMOv3.generate_data(parameters)
end = time.time()

print(f"걸리는시간: {round(end-start,2)}")

The following parameters seem unnecessary:
{'activate_OFDM'}

Scene 1/10

Basestation 1

UE-BS Channels


Generating channels: 100%|█████████████████████████████████████████████████████| 69040/69040 [00:23<00:00, 2972.34it/s]



BS-BS Channels


Generating channels: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.18it/s]



Scene 2/10

Basestation 1

UE-BS Channels


...


BS-BS Channels


Generating channels: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 220.65it/s]



Scene 10/10

Basestation 1

UE-BS Channels


Reading ray-tracing: 100%|████████████████████████████████████████████████████| 69040/69040 [00:05<00:00, 13351.93it/s]


MemoryError: Unable to allocate 1.05 GiB for an array with shape (69040, 1, 32, 64) and data type complex64

In [ ]:
len(dataset)

In [ ]:
dataset[0]

# 사용자 접근 데이터

In [ ]:
user_data = dataset[0][0]['user']
print(user_data.keys())

# 사용자 채널 정보 확인

In [ ]:
# subcarries = 나눈 각각의 주파수 채널
# Channel = H <- 채널 벡터
# 채널 형태
# (user, UE antenna, Bs antenna, subcarrier)
channel = dataset[0][0]['user']['channel']
print(channel.shape)  

In [ ]:
print(dataset[0][0]['user']['channel'][100])

In [ ]:
print(len(dataset[0][0]['user']['channel'][100]))

In [ ]:
print(channel[10000][0][0])

In [ ]:
print(channel[1][0][0])

# 사용자 위치 정보

In [ ]:
location = dataset[0][0]['user']['location']
print(location.shape)      # (사용자 수, 3)
print(location[0:4])         # 첫 번째 사용자의 (x, y, z)

# 경로정보

In [ ]:
paths = dataset[0][0]['user']['paths']
#사용자 수
print(len(paths))
# 첫 번째 사용자 경로 정보
print(paths[0])

# 기지국 정보

In [ ]:
bs_data = dataset[0][0]['basestation']
print(bs_data.keys())


# Scene 및 사용자 수

In [ ]:
for i, scene in enumerate(dataset[0]):
    user_locs = scene['user']['location']
    print(f"Scene {i}: {len(user_locs)} users")

# 채널 수

In [ ]:
len(dataset[0][0]['user']['channel'])

In [ ]:
print(dataset[0][0]['user']['paths'][0])

In [ ]:
scene = dataset[0][0] # scene 0
ue_idx = 0 # 첫 번째 사용자
channel = scene['user']['channel'][ue_idx]
print(channel.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

# channel CIR mat 정보 가져오기

In [ ]:
import scipy.io as sio

file_path = r'C:\Users\dlghd\졸업프로젝트\LWM\O2_dyn_3p5\scene_0\O2_dyn_3p5.1.CIR.mat'
mat_data = sio.loadmat(file_path)

# 파일 안의 key 확인
print(mat_data.keys())




In [ ]:
# 일반적으로 CIR key는 'CIR' 또는 'cir' 같은 이름일 가능성 높음
H_cir = mat_data['__header__']  
print(H_cir)

# Time-Prediction 시작
## Time Series 형태로 변환
### 단일사용자 채널 예측

In [ ]:
# H = dataset[0][0]['user']['channel'][100]  # (1, 32, 64), complex
# H = H.squeeze(0)  # → (32, 64)

# # 채널은 복소수 형태
# H_real = H.real   # shape: (32, 64)
# H_imag = H.imag   # shape: (32, 64)

# H_combined = np.concatenate([H_real, H_imag], axis=0)  # shape: (64, 64)


In [ ]:
# # 단일 사용자에 대한 예측 코드
# channel_sequence = []

# for scene in dataset:
#     # 100번재 사용자에 대한 예측
#     H = scene[0]['user']['channel'][100]
#     H = H.squeeze(0)
#     # 채널은 복소수 형태 
#     H_real = H.real # (32, 64) = (BS antenna, subcarrier)
#     H_imag = H.imag
#     H_concat = np.concatenate([H_real, H_imag], axis=0) 
#     channel_sequence.append(H_concat)

# # T는 sence
# channel_sequence = np.stack(channel_sequence)  # shape: (T, 64, 64) = (real/imag쌍, subcarrier)


In [ ]:

print(dataset[0][0]['user']['channel'].shape)

In [ ]:
# print("-" * 40)
# print("scene 개수 :", len(dataset[0]))

# valid_users = 0
# for s_idx, scene in enumerate(dataset):
#     ch_all = scene['user']['channel']
#     nonzero_mask = np.abs(ch_all).sum(axis=(1,2,3)) > 0   # (num_users,)
#     n_ok = nonzero_mask.sum()
#     print(f"scene {s_idx}  ▶  사용자 {n_ok}/{len(ch_all)} 명 유효")
#     valid_users += n_ok
# print("총 유효 사용자(중복 포함):", valid_users)
# print("-" * 40)


In [ ]:
print(dataset[0][0]['user']['channel'])

In [ ]:
# user 전체 시퀀스 만들어서 전체 user에 대한 prediction 진행하기
# 형태 dict user: H
user_sequence = {}

for scene in dataset:
    channel = scene[0]['user']['channel']  # user,1,32,64
    
    for user, H in enumerate(channel):
        if np.abs(H).sum() == 0:
            continue
        
        H = H.squeeze(0) # 32,64
        H_concat = np.concatenate([H.real,H.imag], axis=0)
        
        user_sequence.setdefault(user,[]).append(H_concat)


In [ ]:
# (scene 수)
print("scene 수:", len(user_sequence[0]))
# 채널 행렬 크기 (64, 64)
print("user_sequence[0][0] shape:", np.array(user_sequence[0][0]).shape)
# 서브캐리어
print("user_sequence[0][0][0] 길이:", len(user_sequence[0][0][0]))


In [ ]:
seq_len = 5
X = []
y = []

# sequence 부족한거 패스
for seq in user_sequence.values():  
    if len(seq) < seq_len:
        continue  
    seq = np.stack(seq)  # (T_user, 64, 64)

    for i in range(len(seq) - seq_len + 1):
        X.append(seq[i:i+seq_len-1])        # (seq_len-1, 64, 64)
        y.append(seq[i+seq_len-1])          # (64, 64)


In [ ]:
print(type(X))

In [87]:
X = np.stack(X)
y = np.stack(y)

MemoryError: Unable to allocate 17.2 GiB for an array with shape (282322, 4, 64, 64) and data type float32

In [84]:
# Flatten
# 4096 = 64 * 64

X_flat = X.reshape(X.shape[0], X.shape[1], -1) # (5,4,4096) = (샘플, 시퀀스 길이)
y_flat = y.reshape(y.shape[0],-1) #(5,4096) # 샘플 대상 예측

AttributeError: 'list' object has no attribute 'reshape'

In [100]:
print(X_flat.shape)
print(y_flat.shape)

(6, 4, 4096)
(6, 4096)


In [101]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 아래 텐서 cpu로
X_t = torch.tensor(X_flat, dtype=torch.float32)        # (N, seq_len, element_length)
y_t = torch.tensor(y_flat, dtype=torch.float32)        # (N,)
masked_pos = torch.full((X_t.shape[0], 1),             # (N, 1)
                        seq_len - 2,
                        dtype=torch.long)

#  Dataset 생성
full_ds = TensorDataset(X_t, masked_pos, y_t)

# train / val 8:2 분할
train_size = int(len(full_ds) * 0.8)
val_size   = len(full_ds) - train_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size])

# DataLoader 정의
batch_size = 32

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,        # epoch마다 섞기
    drop_last=False
)

val_loader = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False
)


In [102]:
print(X_t)

tensor([[[-4.9277e-06, -4.7682e-06, -4.4857e-06,  ..., -5.5699e-06,
          -5.9204e-06, -6.1182e-06],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  .

In [103]:
print(X_t[3][0][1])

tensor(0.)


In [104]:
print(X_t.shape)
print(y_t.shape)
# mini_batch, sequence_len, H_real+H_imag

torch.Size([6, 4, 4096])
torch.Size([6, 4096])


# 아래 코드 구조
┌──────────────────────────────────────────────────────────────┐
│ input_ids  (B, seq_len, element_length)  ─┐                 │
│ masked_pos (B, num_mask)                  ├─>  LWM backbone │
│                                           │    (12-층 트랜스포머)  
└────────────────────────────────────────────┘         │
            logits_lm  (B, num_mask, element_length)  │   enc_output (B, seq_len, d_model)
                                                      ▼
                        ┌─[풀링]───────────────┐      ←── feat (B, d_model)
                        │ 첫 토큰(0번) 선택    │
                        │   or 평균/최대 풀링 │
                        └──────────────────────┘
                                      ▼
                       FC 헤드  (d_model → hidden_dim → out_dim)
                                      ▼
                                out (B, out_dim)

# 시각적비유

[패치 프로젝터]──▶[Transformer ×12]──▶[LayerNorm]──┐
                                                  ├─▶ 64-차 벡터 (CLS 또는 풀링) ─▶ MLP ─▶ out                                                
[Positional Embedding]─────────────────────────────┘


In [105]:
"""
LWMWithHead: 사전학습된 LWM(Transformer encoder)을 ‘백본(backbone)’으로 사용하고,
그 뒤에 새로운 완전연결(FC) 헤드(head)를 붙여 다운스트림 작업(회귀·분류 등)에
사용할 수 있도록 만든 래퍼(wrapper) 클래스입니다.
"""

import torch
import torch.nn as nn
from lwm_model import lwm   # 기존 LWM 모델 클래스 (import 경로는 프로젝트 구조에 맞게 조정)

class LWMWithHead(nn.Module):
    """
    Args
    ----
    element_length : int
        LWM 입력 패치의 길이. 예) 64*64 = 4096 (H_real + H_imag)
    d_model        : int
        Transformer 모델 차원(=LWM hidden size).
    max_len        : int
        포지셔널 임베딩 최대 길이(시퀀스 길이).
    n_layers       : int
        Transformer 인코더 층 수.
    hidden_dim     : int
        새 FC 헤드의 중간 차원.
    out_dim        : int
        최종 출력 차원. 1 → 회귀/이진분류, k → k-클래스 분류.
    freeze_backbone: bool
        True면 백본을 동결(freeze)하여 헤드만 학습.
    ckpt_path      : str | None
        사전학습 가중치(.pth) 경로. None이면 랜덤 초기화.
    device         : str
        'cuda' / 'cpu' 등 모델을 올릴 장치.
    """

    def __init__(
        self,
        element_length: int = 4096,
        d_model: int = 64,
        max_len: int = 129,
        n_layers: int = 12,
        hidden_dim: int = 256,
        out_dim: int = 1,
        freeze_backbone: bool = False,
        ckpt_path: str | None = None,
        device: str = "cuda",
    ):
        super().__init__()

        # ────────────────────────────
        # 1) 백본(backbone) 초기화
        # ────────────────────────────
        if ckpt_path is None:
            # 가중치 없이 새로 생성
            self.backbone = lwm(
                element_length=element_length,
                d_model=d_model,
                max_len=max_len,
                n_layers=n_layers
            ).to(device)
        else:
            # 사전학습 가중치 로드
            self.backbone = lwm.from_pretrained(
                ckpt_name=ckpt_path,
                device=device
            )

        # 백본 동결(선택)
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

        # ────────────────────────────
        # 2) 헤드(head) 정의
        # ────────────────────────────
        self.head = nn.Sequential(
            nn.Linear(d_model, hidden_dim),  # 첫 FC
            nn.ReLU(),                       # 활성화
            nn.Dropout(0.3),                 # 드롭아웃
            nn.Linear(hidden_dim, out_dim)   # 최종 FC
        )

    # ────────────────────────────
    # forward
    # ────────────────────────────
    def forward(self, input_ids, masked_pos):
        """
        Parameters
        ----------
        input_ids : Tensor  (B, seq_len, element_length)
            LWM 입력 시퀀스 (패치/토큰 단위 실수·복소수 채널값 등).
        masked_pos : Tensor  (B, num_mask)
            LWM의 마스크드 채널 모델링용 인덱스 (백본 규격 유지용).

        Returns
        -------
        out : Tensor  (B, out_dim)
            헤드에서 계산된 다운스트림 작업용 로짓/예측값.
        """

        # 기존 LWM forward:
        #   logits_lm : (B, num_mask, element_length)  ← 사용 안 함
        #   enc_output: (B, seq_len, d_model)
        _, enc_output = self.backbone(input_ids, masked_pos)

        # 특징 추출(feat)
        # ① 첫 토큰 벡터 사용 (CLS 개념) ─────────────
        feat = enc_output[:, 0, :]           # (B, d_model)

        # ② 평균 풀링 예시 (필요 시 주석 해제) ─────
        # feat = enc_output.mean(dim=1)       # (B, d_model)

        # ③ Max 풀링 예시 (필요 시 주석 해제) ─────
        # feat, _ = enc_output.max(dim=1)     # (B, d_model)

        # 헤드 통과 → 최종 출력
        out = self.head(feat)                # (B, out_dim)
        return out


In [106]:
import torch
import torch.nn as nn
from torch.optim import Adam

# 디바이스 설정(GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 초기화
element_length = 64 * 64
model = LWMWithHead(
    element_length=element_length,
    out_dim=element_length, # 회귀(스칼라 예측) = out_dim을 4096으로해야 진행됨
    freeze_backbone=False,  # backbone과 같이 학습
    ckpt_path = None, # 사전학습 weight 없으며 None
    device = device
).to(device)

# 손실함수
criterion = nn.MSELoss()

# 옵티마이저 설정
optimizer = Adam(model.parameters(), lr=1e-4)

Using device: cuda


In [107]:
# 모델 학습
import time
start = time.time()

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for input_ids, masked_pos, target in train_loader:
        input_ids, masked_pos, target = [x.to(device) for x in (input_ids, masked_pos, target)]
        
        pred = model(input_ids, masked_pos).squeeze(-1)  # (B,)로 맞춤
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")
end = time.time()

print(f"걸린시간: {round(end-start,2)}")

Epoch 1/10, Loss: 0.099071
Epoch 2/10, Loss: 0.079470
Epoch 3/10, Loss: 0.064805
Epoch 4/10, Loss: 0.048726
Epoch 5/10, Loss: 0.048853
Epoch 6/10, Loss: 0.043881
Epoch 7/10, Loss: 0.039516
Epoch 8/10, Loss: 0.034874
Epoch 9/10, Loss: 0.030161
Epoch 10/10, Loss: 0.033556
걸린시간: 0.5


In [108]:
# 모델 평가 방법
import torch.nn.functional as F

def rmse(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    RMSE = Root MSE
    RMSE = {1/n*sum((y^-y)**2)}**1/2
    """
    return torch.sqrt(F.mse_loss(pred, target, reduction="mean"))   # √MSE

def nmse(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Normalized MSE  =  E[‖ŷ − y‖²] / E[‖y‖²]
    returns: 스칼라 (배치 평균)
    """
    # (B, …) → (B,)  : 각 샘플별 제곱합
    mse_per_sample   = ((pred - target)**2).view(pred.size(0), -1).sum(dim=1)
    power_per_sample = (target**2).view(target.size(0), -1).sum(dim=1)
    return (mse_per_sample / power_per_sample).mean()


In [109]:
# 모델 평가 함수
import torch
import torch.nn.functional as F

# ─────────────────────────────────────────
# 1. 배치 단위 RMSE, NMSE 함수
# ─────────────────────────────────────────
def rmse(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Root-Mean-Squared Error
    returns: 스칼라 (배치 평균)
    """
    return torch.sqrt(F.mse_loss(pred, target, reduction="mean"))   # √MSE

def nmse(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Normalized MSE  =  E[‖ŷ − y‖²] / E[‖y‖²]
      · 채널 예측 분야에서 흔히 쓰는 지표
    returns: 스칼라 (배치 평균)
    """
    # (B, …) → (B,)  : 각 샘플별 제곱합
    mse_per_sample   = ((pred - target)**2).view(pred.size(0), -1).sum(dim=1)
    power_per_sample = (target**2).view(target.size(0), -1).sum(dim=1)
    return (mse_per_sample / power_per_sample).mean()


# ─────────────────────────────────────────
# 2. 검증 루프 예시
# ─────────────────────────────────────────
def evaluate(model, loader, device="cuda"):
    model.eval()
    total_rmse, total_nmse = 0.0, 0.0

    with torch.no_grad():
        for input_ids, masked_pos, target in loader:
            input_ids  = input_ids.to(device)
            masked_pos = masked_pos.to(device)
            target     = target.to(device)

            pred = model(input_ids, masked_pos)

            total_rmse += rmse(pred, target).item() * input_ids.size(0)
            total_nmse += nmse(pred, target).item() * input_ids.size(0)

    N = len(loader.dataset)
    return {
        "RMSE": total_rmse / N,
        "NMSE": total_nmse / N
    }


In [110]:
num_epochs = 30
patience_counter = 0

for epoch in range(1, num_epochs + 1):
    # ──────────── ① 학습 ────────────
    model.train()
    for input_ids, masked_pos, target in train_loader:
        input_ids  = input_ids.to(device)
        masked_pos = masked_pos.to(device)
        target     = target.to(device)

        pred = model(input_ids, masked_pos)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # ──────────── ② 검증 ────────────
    metrics = evaluate(model, val_loader, device)
    rmse_val, nmse_val = metrics["RMSE"], metrics["NMSE"]

    # ──────────── ③ 로그 출력 ────────────
    print(f"[Epoch {epoch:02d}]  "
          f"val_RMSE = {rmse_val:.4f}   "
          f"val_NMSE = {nmse_val:.4e}   "
          f"val_NMSE(dB) = {10*torch.log10(torch.tensor(nmse_val)):.2f} dB")


[Epoch 01]  val_RMSE = 0.1194   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 02]  val_RMSE = 0.1135   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 03]  val_RMSE = 0.1082   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 04]  val_RMSE = 0.1039   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 05]  val_RMSE = 0.1008   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 06]  val_RMSE = 0.0985   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 07]  val_RMSE = 0.0965   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 08]  val_RMSE = 0.0945   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 09]  val_RMSE = 0.0925   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 10]  val_RMSE = 0.0907   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 11]  val_RMSE = 0.0889   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 12]  val_RMSE = 0.0870   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 13]  val_RMSE = 0.0849   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch 14]  val_RMSE = 0.0826   val_NMSE = inf   val_NMSE(dB) = inf dB
[Epoch